In [1]:
import sys
!{sys.executable} -m pip install --upgrade seaborn


In [2]:
import sys
print("Jupyter Python executable:", sys.executable)


Jupyter Python executable: C:\Users\own\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe


In [3]:


# evaluate_model.ipynb - run this cell to evaluate the saved model and save a JSON report

import os
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

# --- 1) Paths (adjust only if needed) ---
# model is in notebooks/eco_model.h5
MODEL_PATH = os.path.join("..", "notebooks", "eco_model.h5")

# use the validation directory that your training used (this is the usual location)
VAL_DIR = os.path.join("..", "dataset", "validation")

# results path
RESULTS_DIR = os.path.join("..", "results")
os.makedirs(RESULTS_DIR, exist_ok=True)
REPORT_PATH = os.path.join(RESULTS_DIR, "evaluation_report.json")

print("Model path:", MODEL_PATH)
print("Validation dir:", VAL_DIR)
print("Results path:", REPORT_PATH)

# --- 2) Load model ---
model = tf.keras.models.load_model(MODEL_PATH)
print("✅ Model loaded.")

# --- 3) Data generator for validation images ---
IMG_SIZE = (150, 150)
BATCH_SIZE = 32

val_datagen = ImageDataGenerator(rescale=1.0/255)
val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# --- 4) Evaluate ---
val_loss, val_acc = model.evaluate(val_generator)
print(f"\nValidation accuracy: {val_acc*100:.2f}%")
print(f"Validation loss: {val_loss:.4f}")

# --- 5) Predictions for report ---
pred_probs = model.predict(val_generator)
y_pred = np.argmax(pred_probs, axis=1)
y_true = val_generator.classes
labels = list(val_generator.class_indices.keys())

# classification report (dict)
report_dict = classification_report(y_true, y_pred, target_names=labels, output_dict=True)
print("\nClassification report (summary):")
for lbl in labels:
    r = report_dict.get(lbl, {})
    print(f"{lbl}: precision={r.get('precision',0):.2f}, recall={r.get('recall',0):.2f}, f1={r.get('f1-score',0):.2f}")

# confusion matrix and plot
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# sample images visualization (optional)
val_generator.reset()
x_batch, y_batch = next(val_generator)
pred_batch = model.predict(x_batch)

plt.figure(figsize=(12,6))
for i in range(min(6, x_batch.shape[0])):
    plt.subplot(2,3,i+1)
    plt.imshow(x_batch[i])
    true_label = labels[np.argmax(y_batch[i])]
    pred_label = labels[np.argmax(pred_batch[i])]
    color = "green" if true_label == pred_label else "red"
    plt.title(f"True: {true_label}\nPred: {pred_label}", color=color)
    plt.axis("off")
plt.tight_layout()
plt.show()

# --- 6) Save JSON report ---
out = {
    "validation_accuracy": float(val_acc),
    "validation_loss": float(val_loss),
    "classification_report": report_dict,
    "labels": labels
}

with open(REPORT_PATH, "w") as f:
    json.dump(out, f, indent=4)

print(f"\n✅ Evaluation report saved to: {REPORT_PATH}")


Model path: ..\notebooks\eco_model.h5
Validation dir: ..\dataset\validation
Results path: ..\results\evaluation_report.json


✅ Model loaded.
Found 200 images belonging to 8 classes.
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 119ms/step - accuracy: 0.3450 - loss: 1.5369

Validation accuracy: 34.50%
Validation loss: 1.5369
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 120ms/step


ValueError: Number of classes, 6, does not match size of target_names, 8. Try specifying the labels parameter

In [ ]:
model.output_shape
